In [ ]:
%pip install -r requirements.txt -q

In [ ]:
%pip install tiktoken -q

# Create Vectorstore
In this notebook we create a vectorstore in our existing Postgressql aurora serverless db.
We will store Zendesk articles existing in our knowledge base. We will also build a way to track existing articles already stored.

## Database Connection

In [ ]:
from utils import db

In [ ]:
engine = db.engine

## Get zendesk articles

In [ ]:
from utils.zendesk_articles import loader
import json
import pandas as pd

In [ ]:
zendesk_loader = loader

In [ ]:
documents = zendesk_loader.load()

In [ ]:
print(f"Nr of articles in the knowledge base {len(documents)}")
documents[:2]

## Create a Log in the database about existing articles and latest updates
1. Extract the article ID and the latest update date
2. Store in a simple tracking table in RDS

In [ ]:
metadata_list = [d.metadata for d in documents]
metadata_list[:2]

In [ ]:
md_df = pd.DataFrame(metadata_list)
md_df["updated_at"] = pd.to_datetime(md_df["updated_at"])
md_df.info(), md_df.head()

In [ ]:
#Store the tracking info
try:
    md_df.to_sql('zendesk_kb_tracker', con=engine, if_exists='append', index=False)
    print("Data inserted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

## Work with embeddings
What is the size of our articles, do we chunk them?

In [ ]:
# Build a dataframe with the doc content and see the article length
docs = [d.to_dict() for d in documents]
doc_list = []
for doc in docs:
    d = {}
    d["id"] = doc["metadata"].get("id", "")
    d["title"] = doc["metadata"].get("title", "")
    d["updated_at"] = doc["metadata"].get("updated_at", "")
    d["text"] = doc.get("text", "")
    doc_list.append(d)
doc_df = pd.DataFrame(doc_list)
doc_df = doc_df.assign(text_len = doc_df["text"].apply(lambda x: len(x.split())))
doc_df
                                          

In [ ]:
doc_df["text_len"].max()

# Embeddings

In [ ]:
len(documents)

In [ ]:
#Convert to langchain format
docs = [doc.to_langchain_format() for doc in documents]
print(len(docs))
docs[0]

In [ ]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock

bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region="eu-central-1",
    runtime=True # Default. Needed for invoke_model() from the data plane
)

In [ ]:
from utils.TokenCounterHandler import TokenCounterHandler

token_counter = TokenCounterHandler()

In [ ]:
from langchain.embeddings import BedrockEmbeddings

embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                               client=bedrock_client)

In [ ]:
from langchain_community.vectorstores.pgvector import PGVector

In [ ]:
COLLECTION_NAME = "zendesk_articles"

## Create Vector Database

In [ ]:
db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=db.connection_str,
)

## If you already have a database, Load it instead

In [ ]:
db =PGVector.from_existing_index(embedding=embeddings,collection_name=COLLECTION_NAME, connection_string=conn_string)

## Test the retrieval mechanism

In [ ]:
query ="What do I need to do when I create a new case?"

In [ ]:
docs_with_score = db.similarity_search_with_score(query,k=2)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

# Try Question answering
Lets try a chain

## First load a LLM

In [ ]:
from langchain.llms.bedrock import Bedrock

model_kwargs_claude = { 
        "max_tokens_to_sample": 512,
        "stop_sequences": [],
        "temperature":0,  
        "top_p":0.5
    }

# Anthropic Claude Model
llm = Bedrock(
    model_id="anthropic.claude-v2", 
    client=bedrock_client, 
    model_kwargs=model_kwargs_claude,
)

In [ ]:
#Test
llm.invoke("Whats up")

## Set retriever from the db

In [ ]:
retriever =db.as_retriever(search_kwargs={"k": 2})

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    callbacks=[token_counter]
)

In [ ]:
result = qa.invoke({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
result["result"]

In [ ]:
print(doc_df[doc_df["title"]=="Log in Issues"]["text"].values[0])

In [ ]:
query = "I have problems with my registration, what are the steps I need to follow?"
result = qa.invoke({"query": query})
print(result['result'])
print(f"\n{result['source_documents']}")